In [18]:
import nltk
import requests
import os
from dotenv import load_dotenv
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
load_dotenv()
NEWS_API_KEY= os.getenv("news_key")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/aadi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [19]:
def fetch_news():
    url = f"https://newsapi.org/v2/everything?q=geopolitics&language=en&pageSize=10&sortBy=publishedAt&apiKey={NEWS_API_KEY}"
    response=requests.get(url)
    data=response.json()
    articles=data.get("articles",[])
    news = [article["title"] + ". " + str(article["description"]) for article in articles]
    return news


In [20]:


def analyze_sentiment(news_list):
    results=[]
    for news in news_list:
        sentiment=sia.polarity_scores(news)
        results.append({
            "news":news,
            "sentiment_score":sentiment["compound"]
        })
    return results

In [21]:
news=fetch_news()
sentiment_result=analyze_sentiment(news)

for item in sentiment_result:
    print(f"##{item['news']}\n  -> Sentiment Score: {item['sentiment_score']}\n")
    

##David McWilliams: ‘I’ve never seen anything like this economic chaos. Buckle up’. Punters and businesses have reacted to the past few weeks of chaos by becoming much less confident about the future
  -> Sentiment Score: -0.7652

##Joe Rogan Squirms As He’s Called Out For Legitimizing ‘Dangerous’ People With ‘S**t’ Ideas. Commentator Douglas Murray grilled the comedian about his “selection of guests,” considering Rogan has the “world’s No. 1 podcast.”
  -> Sentiment Score: 0.1027

##U.S.–ROK Cooperation Can Shape the Nuclear Energy Market and Strengthen Nonproliferation. The relationship between the United States and South Korea has become a strategic necessity for sustaining leadership in the nuclear energy market and preserving the integrity of the global nonproliferation regime. Nuclear geopolitics has been in crisis. The …
  -> Sentiment Score: 0.7003

##Trump's Financial Shakedown: Top Banker Claims America Facing 'Considerable Turbulences' and 'Trade Wars' as Citizens Await Unpr

In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("gem_key")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite-preview-02-05", temperature=0.6)

In [23]:
def get_llm_relevance(news_text):
    prompt=f"Is the following news article likely to influence stock markets?If yes,state yes and explain how.\n\nArticle:{news_text}"
    response=llm.invoke(prompt)
    return response.content.strip()

In [24]:
def filter(news_list):
    relevent_articles=[]
    for article in news_list:
        relevance_score=get_llm_relevance(article)
        if "yes" in relevance_score.lower():
            relevent_articles.append(article)
    return relevent_articles

In [25]:
import pandas as pd
from datetime import datetime

In [28]:
from statistics import mode


def process_and_save(filename="sentiment.csv"):
    news_list=fetch_news()
    relevant_articles=filter(news_list)

    data=[]

    for news in relevant_articles:
        sentiment=sia.polarity_scores(news)
        relevance_explanation=get_llm_relevance(news)

        data.append({
            "timestamp":datetime.now().isoformat(),
            "article":news,
            "sentiment_score":sentiment["compound"],
            "label":"Positive" if sentiment["compound"]>0.2 else "Negative",
            "explanation":relevance_explanation
        })

    df=pd.DataFrame(data)

    writer_header=not os.path.exists(filename)
    df.to_csv(filename,mode='a',index=False,header=writer_header)

In [29]:
process_and_save()